In [39]:
from dotenv import load_dotenv
import urllib.parse as urlparse
import requests
import json
import os

class Untappd:

    def __init__(self, clientId = "", clientSecret = ""):
        load_dotenv(dotenv_path="../.env")
        self._clientId = os.getenv("CLIENT_ID_ALT")
        self._clientSecret = os.getenv("CLIENT_SECRET_ALT")
        self._baseUrl = "https://api.untappd.com/v4/"


    def keys(self):
        return {
            "client_id": self._clientId,
            "client_secret": self._clientSecret
        }

    def findBrewery(self, brewery="", writeFile=False):
        methodUrl = self._baseUrl + "search/brewery"
        params = {
            "q": brewery
        }
        requestUrl = methodUrl + "?" + urlparse.urlencode({**params, **self.keys()})
        response = requests.get(requestUrl)
        
        headers = response.headers
        counter = int(headers["X-Ratelimit-Remaining"])
        breweries = [] 
        
        if response.status_code == 200:
            content = response.json()
            
            
            if content["response"]["found"] > 0:
                breweries = content["response"]["brewery"]["items"]
        
        return breweries, counter, response
    
    def breweryInfo(self, breweryId="", writeFile=False):
        methodUrl = self._baseUrl + "brewery/info/"
        requestUrl = methodUrl + str(breweryId) + "?" + urlparse.urlencode(self.keys())
        response = requests.get(requestUrl)
        
        headers = response.headers
        counter = int(headers["X-Ratelimit-Remaining"])
        
        if response.status_code == 200:
            content = response.json()
            
            brewery = content["response"]
        
        return brewery, counter, response




In [40]:
import numpy as np
import pandas as pd

untappd = Untappd()

ntx_breweries = pd.read_csv("../data/NTX Brewery Info.csv")

breweries = ntx_breweries.Brewery.tolist()

brewery_search = []
for brewery in breweries:
    breweries, counter, response = untappd.findBrewery(brewery)
    brewery_search.append(breweries)
    
    print(counter)
    if counter < 0:
        break

# breweries

100
99
98
97
96
95
94
93
92
91
90
89
88
87
86
85
84
83
82
81
80
79
78
77
76
75
74
73
72
71
70
69
68
67
66
65
64
63
62
61
60
59
58
57
56
55
54
53
52
51
50
49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16


In [67]:

# Normalized the results
brewery_parsed = []
for entries in brewery_search:
    for brewery in entries:
        brewery_parsed.append({
            "search_count":len(entries),
            "brewery_name":brewery["brewery"]["brewery_name"],
            "brewery_id":brewery["brewery"]["brewery_id"],
            "beer_count":brewery["brewery"]["beer_count"],
            "brewery_city":brewery["brewery"]["location"]["brewery_city"],
            "brewery_state":brewery["brewery"]["location"]["brewery_state"]
        })
#         print(brewery)
#         print("##########")
#     break
        

# Remove results where not in texas
brewery_parsed = [ brewery for brewery in brewery_parsed if "TX" in brewery["brewery_state"]]
# Must have 10 beers!
brewery_parsed = [ brewery for brewery in brewery_parsed if brewery["beer_count"] > 9 ]

#Convert to Pandas and save it
brewery_parsed = pd.DataFrame( brewery_parsed )
brewery_parsed.to_csv("../data/brewery_search.csv")
pd.options.display.max_rows = 999
display( brewery_parsed )
pd.options.display.max_rows = 15

,search_count,brewery_name,brewery_id,beer_count,brewery_city,brewery_state
0,1,3 Nations Brewing,205645,42,Farmers Branch,TX
1,1,903 Brewers,45007,102,Sherman,TX
2,1,Armadillo Ale Works™,9316,28,Denton,TX
3,1,Bankhead Brewing Co.,293554,44,Rowlett,TX
4,1,Barley & Board,221680,31,Denton,TX
5,1,Bishop Cider Company,144590,207,Dallas,TX
6,1,Bitter Sisters Brewing Company,72194,41,Addison,TX
7,1,Bluffview Growler,419505,12,Dallas,TX
8,2,BrainDead Brewing,185850,227,Dallas,TX
9,1,Brutal Beerworks,178427,59,North Richland Hills,TX


4

In [3]:
breweryId = breweries[0]["brewery"]["brewery_id"]

untappd = UntappdApi(os.getenv("CLIENT_ID"),os.getenv("CLIENT_SECRET"))
brewery, counter, response = untappd.breweryInfo(breweryId)

brewery

{'brewery': {'brewery_id': 48372,
  'brewery_name': 'Community Beer Company',
  'brewery_slug': 'community-beer-company',
  'brewery_page_url': '/CommunityBeerCompanyTexas',
  'brewery_label': 'https://untappd.akamaized.net/site/brewery_logos/brewery-48372_9a99a.jpeg',
  'brewery_label_hd': '',
  'country_name': 'United States',
  'brewery_in_production': 0,
  'is_independent': 1,
  'claimed_status': {'is_claimed': True,
   'claimed_slug': 'CommunityBeerCompanyTexas',
   'follow_status': False,
   'follower_count': 10003,
   'uid': 514817,
   'mute_status': 'none'},
  'beer_count': 213,
  'contact': {'twitter': 'CommunityBeerCo',
   'facebook': 'http://www.facebook.com/communitybeer',
   'instagram': 'communitybeerco',
   'url': 'http://www.communitybeer.com'},
  'brewery_type': 'Regional Brewery',
  'brewery_type_id': 11,
  'location': {'brewery_address': '1530 Inspiration Dr',
   'brewery_city': 'Dallas',
   'brewery_state': 'TX',
   'brewery_lat': 32.7909,
   'brewery_lng': -96.8158

In [59]:
entries

[{'brewery': {'brewery_id': 222679,
   'beer_count': 76,
   'brewery_name': 'Westlake Brewing Company',
   'brewery_slug': 'westlake-brewing-company',
   'brewery_page_url': '/WestlakeBrewingCompany',
   'brewery_label': 'https://untappd.akamaized.net/site/brewery_logos/brewery-222679_50302.jpeg',
   'country_name': 'United States',
   'location': {'brewery_city': 'Westlake Village',
    'brewery_state': 'CA',
    'lat': 34.1541,
    'lng': -118.802}}},
 {'brewery': {'brewery_id': 305108,
   'beer_count': 29,
   'brewery_name': 'Westlake Brewing Company',
   'brewery_slug': 'westlake-brewing-company',
   'brewery_page_url': '/Westlake_Brewing_Company',
   'brewery_label': 'https://untappd.akamaized.net/site/brewery_logos/brewery-305108_24443.jpeg',
   'country_name': 'United States',
   'location': {'brewery_city': 'Dallas',
    'brewery_state': 'TX',
    'lat': 32.7831,
    'lng': -96.7824}}},
 {'brewery': {'brewery_id': 196104,
   'beer_count': 10,
   'brewery_name': 'West Lake Brewi

In [19]:
from dotenv import load_dotenv
import os

untappd = Untappd()

breweries, counter, response = untappd.findBrewery("Community Beer Company")

breweries

[{'brewery': {'brewery_id': 48372,
   'beer_count': 213,
   'brewery_name': 'Community Beer Company',
   'brewery_slug': 'community-beer-company',
   'brewery_page_url': '/CommunityBeerCompanyTexas',
   'brewery_label': 'https://untappd.akamaized.net/site/brewery_logos/brewery-48372_9a99a.jpeg',
   'country_name': 'United States',
   'location': {'brewery_city': 'Dallas',
    'brewery_state': 'TX',
    'lat': 32.7909,
    'lng': -96.8158}}},
 {'brewery': {'brewery_id': 259927,
   'beer_count': 1,
   'brewery_name': 'The Community Beer Company',
   'brewery_slug': 'the-community-beer-company',
   'brewery_page_url': '/w/the-community-beer-company/259927',
   'brewery_label': 'https://untappd.akamaized.net/site/assets/images/temp/badge-brewery-default.png',
   'country_name': 'United States',
   'location': {'brewery_city': '', 'brewery_state': '', 'lat': 0, 'lng': 0}}}]